In [1]:
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
import pickle

In [2]:
import tensorflow as tf
from tensorflow.python import keras

In [3]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [ ]:
!pip install tensorflow_addons -U

In [4]:
from keras.models import Sequential, Model
from tensorflow.keras.layers import Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
import keras as k
# from keras_contrib.layers import CRF
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import Adam, Nadam
from tensorflow_addons.text import CRFModelWrapper
# from keras_contrib.layers import CRF
# from tensorflow_addons.layers import CRF
# from tf_crf_layer.loss import crf_loss
# from tf_crf_layer.metrics import crf_accuracy

C:\Users\PC\anaconda3\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [5]:
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras.layers import LSTM, Bidirectional, Conv1D, Dense, Dropout, Softmax, MaxPooling1D, Input, Embedding, TimeDistributed
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow_addons.text import CRFModelWrapper
from sklearn.metrics import classification_report, confusion_matrix

In [6]:
print(f"Tensorflow version {tf.__version__}")
print(f"Tensorflow addons version {tfa.__version__}")

Tensorflow version 2.12.0
Tensorflow addons version 0.20.0


In [7]:
data = pd.read_pickle('data_entities_100_.pickle')

In [8]:
data

,text,tag,text_ind,tag_ind
0,"[субсидия, предоставляться, в, предел, бюджетн...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[10, 123, 2, 481, 83, 574, 3, 366, 83, 138, 1,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,"[основной, понятие, ,, использовать, в, настоя...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[854, 549, 1, 384, 2, 34, 13, 53, 170, 4, 35, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,"[доступ, к, функция, сервис, для, участник, от...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[1653, 50, 400, 2198, 37, 27, 11, 339, 1279, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,"[;, участник, отбор, -, представить, в, уполно...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[8, 27, 11, 7, 428, 2, 326, 100, 170, 4, 35, 5...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,"[отбор, получатель, субсидия, проводиться, упо...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[11, 36, 10, 661, 326, 100, 562, 334, 170, 4, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
...,...,...,...,...
12330,"[к, категория, получатель, субсидия, в, рамка,...","[O, O, O, O, i-e-org-ul, O, O, O, O, i-e-org-u...","[50, 243, 36, 10, 2, 236, 34, 13, 503, 22, 16,...","[1, 1, 1, 1, 3, 1, 1, 1, 1, 3, 3, 4, 4, 3, 3, ..."
12331,"[;, 2, ), получатель, субсидия, -, юридический...","[i-e-org-ul, i-e-org-ul, i-e-org-ul, i-e-treb-...","[8, 45, 5, 36, 10, 7, 22, 16, 1, 12, 199, 2, 1...","[3, 3, 3, 9, 3, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1, ..."
12332,"[;, (, в, ред, ., постановление, правительство...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[8, 4, 2, 141, 9, 62, 72, 393, 71, 32, 439, 9,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
12333,"[;, 5, ), представить, в, министерство, (, в, ...","[i-e-treb-sub, i-e-treb-sub, O, O, O, O, O, O,...","[8, 133, 5, 428, 2, 42, 4, 2, 78, 1, 132, 42, ...","[6, 6, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [9]:
with open('dict_words_100_.pkl', 'rb') as dict_words:
    dict_words = pickle.load(dict_words)

In [10]:
with open('dict_tegs_100_.pkl', 'rb') as dict_tegs:
    dict_tegs = pickle.load(dict_tegs)

In [11]:
len(dict_tegs)

57

In [12]:
n_token = len(dict_words)
n_token

6052

In [13]:
n_tag = len(dict_tegs)
n_tag

57

In [14]:
maxlen = max([len(s) for s in  data['text_ind'].tolist()])
maxlen

1725

In [21]:
def get_pad_train_test_val(data):

    #Pad tokens   
    tokens = data['text_ind'].tolist()
#     maxlen = max([len(s) for s in  data['text_ind'].tolist()])
    maxlen =100
    pad_tokens = pad_sequences(tokens, maxlen=maxlen, dtype='int32', padding='post', value= n_token - 1)

    #Pad Tags
    tags = data['tag_ind'].tolist()
    pad_tags = pad_sequences(tags, maxlen=maxlen, dtype='int32', padding='post', value= dict_tegs["o"])
    n_tags = len(dict_tegs)+1
    pad_tags = [to_categorical(i, num_classes=n_tags) for i in pad_tags]
    
    #Split train, test and validation set
    tokens_, test_tokens, tags_, test_tags = train_test_split(pad_tokens, pad_tags, test_size=0.1, train_size=0.9, random_state=2020)
    train_tokens, val_tokens, train_tags, val_tags = train_test_split(tokens_,tags_,test_size = 0.25,train_size =0.75, random_state=2020)

    print(
        'train_tokens length:', len(train_tokens),
        '\ntrain_tags length:', len(train_tags),
        '\ntest_tokens length:', len(test_tokens),
        '\ntest_tags:', len(test_tags),
        '\nval_tokens:', len(val_tokens),
        '\nval_tags:', len(val_tags),
    )
    
    return train_tokens, val_tokens, test_tokens, train_tags, val_tags, test_tags

In [22]:
sum=0
all=0
for i in range(len(data["tag"])):
    count=0
    for num in data["tag"][i]:
        all +=1
        if num == "O":
              count+=1

    sum = sum + count
print(sum)
print(all)
print(sum/all)

408980
682371
0.5993513792350496


In [23]:
train_tokens, val_tokens, test_tokens, train_tags, val_tags, test_tags = get_pad_train_test_val(data)

ValueError: Found array with dim 3. None expected <= 2.

In [ ]:
# фиксируем состояния для воспроизводимости экспериментов
from numpy.random import seed
tf.random.set_seed(2)

In [19]:
input_dim = n_token+1
output_dim = 8
# input_length = max([len(s) for s in  data['text_ind'].tolist()])
input_length = 100
n_tags = len(dict_tegs)+1
print('input_dim: ', input_dim, '\noutput_dim: ', output_dim, '\ninput_length: ', input_length, '\nn_tags: ', n_tags)

input_dim:  6053 
output_dim:  8 
input_length:  100 
n_tags:  58


In [20]:
from keras.models import Model
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional, Activation, Conv1D,Conv2D
import keras as k

inp = Input(shape=(input_length,))

# Embedding Layer
x = Embedding(input_dim=input_dim, output_dim=output_dim, input_length=input_length)(inp)

x = Dropout(0.2)(x)

# BI-LSTM Layer
x_lstm = Bidirectional(LSTM(units=output_dim*2, 
                           return_sequences=True, 
                           dropout=0.2, 
                           recurrent_dropout=0.2, 
                           kernel_initializer=k.initializers.he_normal()))(x)

x_lstm = Bidirectional(LSTM(units=output_dim, 
                           return_sequences=True, 
                           dropout=0.2, 
                           recurrent_dropout=0.2, 
                           kernel_initializer=k.initializers.he_normal()))(x_lstm)

# x_conv3 = Conv1D(filters =128, kernel_size=3, padding='same', kernel_initializer='he_uniform')(x_lstm)
# # avg_pool1_lstm = GlobalAveragePooling1D()(x_conv3)
# # max_pool1_lstm = GlobalMaxPooling1D()(x_conv3)


# x_conv4 = Conv1D(filters =128, kernel_size=2, padding='same', kernel_initializer='he_uniform')(x_lstm)
# avg_pool2_lstm = GlobalAveragePooling1D()(x_conv4)
# max_pool2_lstm = GlobalMaxPooling1D()(x_conv4)

lstm1 = LSTM(64, return_sequences=True)(x_lstm)

# x = Dense(128, activation='relu')(x)
x = Dropout(0.2)(lstm1)


# TimeDistributed Layer
x = TimeDistributed(Dense(n_tags, activation="softmax"))(x)  


model = Model(inputs=inp, outputs=x)

In [21]:
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
BATCH_SIZE=32
#Optimiser 
nadam = tf.keras.optimizers.legacy.Nadam(lr=0.0003, beta_1=0.9, beta_2=0.999)

# Compile model
model.compile(loss='categorical_crossentropy', optimizer=nadam, metrics=['categorical_accuracy'])

model.build((BATCH_SIZE, input_length,))

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100)]             0         
                                                                 
 embedding (Embedding)       (None, 100, 8)            48424     
                                                                 
 dropout (Dropout)           (None, 100, 8)            0         
                                                                 
 bidirectional (Bidirectiona  (None, 100, 32)          3200      
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 100, 16)          2624      
 nal)                                                            
                                                                 
 lstm_2 (LSTM)               (None, 100, 64)           20736 

C:\Users\PC\anaconda3\lib\site-packages\keras\optimizers\legacy\nadam.py:89: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [27]:
epochs = 50
history = model.fit(train_tokens, np.array(train_tags), validation_data=(val_tokens, np.array(val_tags)), epochs=epochs, batch_size=BATCH_SIZE)

Epoch 1/50
261/261 [==============================] - 44s 170ms/step - loss: 0.7206 - categorical_accuracy: 0.8332 - val_loss: 0.7120 - val_categorical_accuracy: 0.8371
Epoch 2/50
261/261 [==============================] - 50s 191ms/step - loss: 0.7138 - categorical_accuracy: 0.8335 - val_loss: 0.7081 - val_categorical_accuracy: 0.8371
Epoch 3/50
261/261 [==============================] - 52s 199ms/step - loss: 0.7115 - categorical_accuracy: 0.8341 - val_loss: 0.7087 - val_categorical_accuracy: 0.8359
Epoch 4/50
261/261 [==============================] - 55s 210ms/step - loss: 0.7060 - categorical_accuracy: 0.8337 - val_loss: 0.7000 - val_categorical_accuracy: 0.8362
Epoch 5/50
261/261 [==============================] - 52s 200ms/step - loss: 0.7016 - categorical_accuracy: 0.8347 - val_loss: 0.7000 - val_categorical_accuracy: 0.8368
Epoch 6/50
261/261 [==============================] - 52s 200ms/step - loss: 0.6966 - categorical_accuracy: 0.8354 - val_loss: 0.6935 - val_categorical_acc

In [ ]:
len(train_tags)

In [ ]:
class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        """
        Keras Layer that implements an Attention mechanism for temporal data.
        Supports Masking.
        Follows the work of Raffel et al. [https://arxiv.org/abs/1512.08756]
        # Input shape
            3D tensor with shape: `(samples, steps, features)`.
        # Output shape
            2D tensor with shape: `(samples, features)`.
        :param kwargs:
        Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
        The dimensions are inferred based on the output shape of the RNN.
        Example:
            model.add(LSTM(64, return_sequences=True))
            model.add(Attention())
        """
        self.supports_masking = True
        #self.init = initializations.get('glorot_uniform')
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        # eij = K.dot(x, self.W) TF backend doesn't support it

        # features_dim = self.W.shape[0]
        # step_dim = x._keras_shape[1]

        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)), K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
    #print weigthted_input.shape
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        #return input_shape[0], input_shape[-1]
        return input_shape[0],  self.features_dim

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=input_dim, output_dim=output_dim, input_length=input_length))

model.add(Conv1D(250, 5, padding='same', activation='relu'))
model.add(GlobalAveragePooling1D())
# model.add(GRU(64, return_sequences=True))

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(n_tags, activation = "softmax"))

nadam = tf.keras.optimizers.legacy.Nadam(lr=0.001, beta_1=0.9, beta_2=0.999)
model.compile(loss='categorical_crossentropy', optimizer=nadam, metrics=['categorical_accuracy'])
history = model.fit(train_tokens, np.array(train_tags), batch_size=256, epochs=10, validation_split=0.1, verbose=1)

In [ ]:
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Conv1D, GRU, CuDNNGRU, CuDNNLSTM, BatchNormalization
from keras.layers import Bidirectional, GlobalMaxPool1D, MaxPooling1D, Add, Flatten, Masking
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, SpatialDropout1D
from keras.models import Model, load_model

In [ ]:
# from keras.models import Model
# from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
# import keras as k
# # from keras_contrib.layers import CRF

# input = Input(shape=(input_length,))

# # Embedding Layer
# model = Embedding(input_dim=input_dim, output_dim=output_dim, input_length=input_length)(input)

# # BI-LSTM Layer
# model = Bidirectional(LSTM(units=output_dim, 
#                            return_sequences=True, 
#                            dropout=0.2, 
#                            recurrent_dropout=0.2, 
#                            kernel_initializer=k.initializers.he_normal()))(model)

# model = LSTM(units=output_dim * 2, 
#              return_sequences=True, 
#              dropout=0.2, 
#              recurrent_dropout=0.2, 
#              kernel_initializer=k.initializers.he_normal())(model)

# # TimeDistributed Layer
# model = TimeDistributed(Dense(n_tags, activation="softmax"))(model)  

# # # CRF Layer
# # crf = CRF(n_tags)

# # out = crf(model)  # output


# model = Model(input, model)

In [ ]:
BATCH_SIZE = 64

In [ ]:
np.array(train_tokens).shape

In [ ]:
np.array(train_tokens).shape

In [ ]:
np.array(train_tags).shape

In [ ]:
train_tokens = tf.reshape(train_tokens , (4481,1000,1))

In [ ]:
# from keras.models import Model
# from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
# import keras as k
# from keras_contrib.layers import CRF

inputs = Input(shape=(input_length,), dtype="int32")

# Embedding Layer
x_embed = Embedding(input_dim=input_dim, output_dim=output_dim, mask_zero=False, input_length=input_length)(inputs)


# BI-LSTM Layer
x_lstm_1 = Bidirectional(LSTM(units=output_dim, 
                           return_sequences=True, 
                           dropout=0.2, 
                           recurrent_dropout=0.2, 
                           kernel_initializer=k.initializers.he_normal()))(x_embed)

outputs = Bidirectional(LSTM(units=output_dim, 
                           return_sequences=True, 
                           dropout=0.2, 
                           recurrent_dropout=0.2, 
                           kernel_initializer=k.initializers.he_normal()))(x_lstm_1)

# outputs = LSTM(units=output_dim * 2, 
#              return_sequences=True, 
#              dropout=0.2, 
#              recurrent_dropout=0.2, 
#              kernel_initializer=k.initializers.he_normal())(x_lstm_1)


# TimeDistributed Layer 

# # TimeDistributed Layer
outputs = TimeDistributed(Dense(256, activation="relu"))(outputs)  
outputs = TimeDistributed(Dropout(0.4))(outputs)

base_model = Model(inputs=inputs, outputs=outputs, name="bi_lstm")

model = CRFModelWrapper(base_model, units=n_tags)


# model = Model(inputs, model)

In [ ]:
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt

#Optimiser 
nadam = tf.keras.optimizers.legacy.Nadam(lr=0.001, beta_1=0.9, beta_2=0.999)

# Compile model
model.compile(loss='categorical_crossentropy', optimizer=nadam, metrics=['categorical_accuracy'])

model.build((BATCH_SIZE, input_length,))

model.summary()

In [ ]:
def false_alarm1(y_true, y_pred): 
    k = 2
    new_y_pred = tf.zeros_like(y_pred)
    _ , indices = tf.math.top_k(y_pred, k = k)
    batch_indices = tf.repeat(tf.range(tf.shape(y_pred)[0]), repeats=k)
    indices = tf.stack([batch_indices, tf.reshape(indices,[-1])], axis=1)
    new_y_pred = tf.tensor_scatter_nd_update(new_y_pred, indices, tf.ones(tf.reduce_prod(tf.shape(indices)[0])))
    print(new_y_pred)

    return tf.math.confusion_matrix(tf.reshape(y_true,[-1]), tf.reshape(new_y_pred,[-1]))

print(false_alarm1(y_true, y_pred))

In [ ]:
inputs.shape

In [ ]:
outputs.shape

In [ ]:
epochs = 10
history = model.fit(train_tokens, np.array(train_tags), validation_data=(val_tokens, np.array(val_tags)), epochs=epochs, batch_size=BATCH_SIZE)

In [ ]:
# history = model.fit(train_tokens, np.array(train_tags), batch_size=256, epochs=80, validation_split=0.1, verbose=1)

In [ ]:
model.save_weights("weights")

In [ ]:
model = keras.models.load_model('model')

In [ ]:
# Plot the graph 
plt.style.use('ggplot')

def plot_history(history):
    accuracy = history.history['categorical_accuracy']
    val_accuracy = history.history['val_categorical_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    x = range(1, len(accuracy) + 1)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(x, accuracy, 'b', label='Training acc')
    plt.plot(x, val_accuracy, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(x, loss, 'b', label='Training loss')
    plt.plot(x, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()

plot_history(history)

In [ ]:
import pickle

In [ ]:
with open('test_labels_all.data', 'rb') as filehandle:  
    test_labels = pickle.load(filehandle)

In [ ]:
with open('pred_labels_all.data', 'rb') as filehandle:  
    pred_labels = pickle.load(filehandle)

In [28]:
key_list = list(dict_tegs.keys())
val_list = list(dict_tegs.values())

def pred2label(pred):
    out = []
    for pred_i in pred:
            out_i = []
            for p in pred_i:
                p_i = np.argmax(p)
                position = val_list.index(p_i)
                out_i.append(key_list[position])
            out.append(out_i)
            
    return out
test_pred = model.predict(test_tokens, verbose=1)   
pred_labels = pred2label(test_pred)
test_labels = pred2label(test_tags)

39/39 [==============================] - 1s 28ms/step


In [ ]:
with open('test_labels_all.data', 'wb') as filehandle:
    pickle.dump(test_labels, filehandle)

In [ ]:
with open('pred_labels_all.data', 'wb') as filehandle:
    pickle.dump(pred_labels, filehandle)

In [ ]:
with open('dict_tegs_full.pkl', 'rb') as dict_tegs:
    dict_tegs = pickle.load(dict_tegs)

In [ ]:
data = pd.read_pickle('data_entities_full.pickle')

In [ ]:
dict_=[]
for i in dict_tegs:
    for j in data["tag"]:
        for k in j:
            if i == k:
                count+=1
    dict_.append({i: count})
    count=0


In [ ]:
dict_

In [ ]:
dict_new=[]
list_tegs=[]
for i in dict_:
    for key, value in i.items():

        if value >=100 or key == 'o':
            dict_new.append({key: value})
            if key !='o':
                list_tegs.append(key[2:])
            else: list_tegs.append(key)


In [ ]:
dict_new

In [ ]:
len(list_tegs)

In [ ]:
for i in list_tegs:
    list_tegs[i] = i

list_tegs[2][2:]

In [ ]:
dict_new

In [ ]:
with open('list_tegs100.txt', 'w') as filehandle:  
    for listitem in list_tegs:
        filehandle.write('%s\n' % listitem)

In [ ]:
places=[]
# откроем файл и считаем его содержимое в список
with open('listfile.txt', 'r') as filehandle:
    for line in filehandle:
        # удалим заключительный символ перехода строки
        currentPlace = line[:-1]

        # добавим элемент в конец списка
        places.append(currentPlace)

In [ ]:
list_tegs

In [ ]:
data

In [29]:
from sklearn.metrics import f1_score, classification_report

In [30]:
def get_score(y_true, y_pred):
    joined_true = []
    joined_pred = []
    for sent_true, sent_pred in zip(y_true, y_pred):
        joined_true.extend(sent_true)
        joined_pred.extend(sent_pred)
        
    print(classification_report(joined_true, joined_pred))
#     print(support(joined_true, joined_pred))

In [31]:
get_score(test_labels, pred_labels)

C:\Users\PC\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\PC\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                            precision    recall  f1-score   support

            i-e-block-treb       0.00      0.00      0.00       129
                   i-e-obj       0.00      0.00      0.00       283
                 i-e-okved       0.00      0.00      0.00        32
            i-e-okved-main       0.00      0.00      0.00       109
         i-e-org-coop-coop       0.00      0.00      0.00        56
         i-e-org-coop-spok       0.00      0.00      0.00       204
                i-e-org-ip       0.00      0.00      0.00       260
               i-e-org-kfl       0.00      0.00      0.00        94
               i-e-org-lph       0.00      0.00      0.00        99
                i-e-org-sf       0.00      0.00      0.00        93
                i-e-org-ul       0.43      0.06      0.11      2612
             i-e-prod-cred       0.56      0.23      0.32       472
              i-e-prod-liz       0.00      0.00      0.00       160
            i-e-prod-strax       0.00      0.00

C:\Users\PC\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
joined_train = []
# joined_pred = []

for i in train_tags:
    joined_train.extend(list(i))

In [ ]:
len(set(joined_train))

In [ ]:
joined_train[0]

In [ ]:
joined_true = []
joined_pred = []
for sent_true, sent_pred in zip(test_labels, pred_labels):
    joined_true.extend(sent_true)
    joined_pred.extend(sent_pred)

In [ ]:
len(set(joined_pred))

In [ ]:
len(set(joined_true))